# [2356. Number of Unique Subjects Taught by Each Teacher](https://leetcode.com/problems/number-of-unique-subjects-taught-by-each-teacher/description/?envType=study-plan-v2&envId=top-sql-50)

Table: Teacher

<pre>+-------------+------+
| Column Name | Type |
+-------------+------+
| teacher_id  | int  |
| subject_id  | int  |
| dept_id     | int  |
+-------------+------+</pre>
(subject_id, dept_id) is the primary key (combinations of columns with unique values) of this table.
Each row in this table indicates that the teacher with teacher_id teaches the subject subject_id in the department dept_id.


Write a solution to calculate the number of unique subjects each teacher teaches in the university.

Return the result table in any order.

The result format is shown in the following example.



Example 1:

Input:
Teacher table:
<pre>+------------+------------+---------+
| teacher_id | subject_id | dept_id |
+------------+------------+---------+
| 1          | 2          | 3       |
| 1          | 2          | 4       |
| 1          | 3          | 3       |
| 2          | 1          | 1       |
| 2          | 2          | 1       |
| 2          | 3          | 1       |
| 2          | 4          | 1       |
+------------+------------+---------+</pre>
Output:  
<pre>+------------+-----+
| teacher_id | cnt |
+------------+-----+
| 1          | 2   |
| 2          | 4   |
+------------+-----+</pre>
Explanation:
Teacher 1:
  - They teach subject 2 in departments 3 and 4.
  - They teach subject 3 in department 3.
Teacher 2:
  - They teach subject 1 in department 1.
  - They teach subject 2 in department 1.
  - They teach subject 3 in department 1.
  - They teach subject 4 in department 1.

In [1]:
# Pandas schema

import pandas as pd

data = [[1, 2, 3], [1, 2, 4], [1, 3, 3], [2, 1, 1], [2, 2, 1], [2, 3, 1], [2, 4, 1]]
teacher = pd.DataFrame(data, columns=['teacher_id', 'subject_id', 'dept_id']).astype(
    {'teacher_id': 'Int64', 'subject_id': 'Int64', 'dept_id': 'Int64'})

#converting to spark dataframe
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

teacher_df = spark.createDataFrame(teacher)


In [2]:
teacher

,teacher_id,subject_id,dept_id
0,1,2,3
1,1,2,4
2,1,3,3
3,2,1,1
4,2,2,1
5,2,3,1
6,2,4,1


In [9]:
# solving in spark dataframe
teacher_df.groupBy('teacher_id').agg(count_distinct('subject_id').alias('cnt'))\
    .show()

+----------+---+
|teacher_id|cnt|
+----------+---+
|         1|  2|
|         2|  4|
+----------+---+



In [8]:
# solving in spark SQL

teacher_df.createOrReplaceTempView('teacher')

spark.sql('''
select
    teacher_id,
    count(distinct subject_id) as cnt
from teacher
group by teacher_id
''').show()

+----------+---+
|teacher_id|cnt|
+----------+---+
|         1|  2|
|         2|  4|
+----------+---+



In [ ]:
spark.stop()